In [6]:
# For colab
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/Github/KT_Devchall"

Mounted at /content/Mydrive
/content/Mydrive/MyDrive/Github/KT_Devchall


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
df_train = pd.read_pickle('data/train.pkl') # 7s

In [9]:
num_features = ['Ex_Rate','P1','P2','P3','P4']
cat_features = df_train.columns.drop(num_features + ['weekend','Class'])

In [10]:
# TO dummy
df_train = pd.get_dummies(df_train, columns=cat_features)

In [11]:
# correlation heatmap
import seaborn as sns
df_corr = df_train.corr()
df_corr

,Ex_Rate,P1,P2,P3,P4,weekend,Class,ADID_type_1,ADID_type_2,ADID_type_5,...,hour_21,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
Ex_Rate,1.000000,0.125311,0.158679,0.161411,0.125189,0.702662,0.111406,-0.140940,0.050147,0.131556,...,-0.009760,-0.000003,0.004495,0.057682,-0.323480,-0.564041,-0.153584,0.125295,0.832906,0.048624
P1,0.125311,1.000000,0.848253,0.856646,0.999616,-0.003446,0.155970,-0.527145,0.284579,0.459666,...,-0.019302,-0.022452,-0.010754,-0.088841,-0.112484,-0.057108,0.076163,0.161771,0.095960,-0.112570
P2,0.158679,0.848253,1.000000,0.986888,0.851888,0.018168,0.148661,-0.579750,0.344046,0.495672,...,-0.020535,-0.015989,0.005096,-0.130321,-0.133463,-0.066688,0.091893,0.187233,0.130209,-0.120761
P3,0.161411,0.856646,0.986888,1.000000,0.859313,0.021331,0.181787,-0.618393,0.286140,0.554642,...,-0.020707,-0.015432,0.006840,-0.131439,-0.137037,-0.065392,0.091819,0.186716,0.133576,-0.120115
P4,0.125189,0.999616,0.851888,0.859313,1.000000,-0.004175,0.153553,-0.525342,0.284414,0.457849,...,-0.019567,-0.022805,-0.010918,-0.089320,-0.112948,-0.057422,0.076332,0.163540,0.095222,-0.112761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dayofweek_2,-0.564041,-0.057108,-0.066688,-0.065392,-0.057422,-0.236202,-0.074930,0.054116,-0.018063,-0.050425,...,0.011743,0.003865,-0.002898,-0.169640,-0.162757,1.000000,-0.185741,-0.198969,-0.166856,-0.143366
dayofweek_3,-0.153584,0.076163,0.091893,0.091819,0.076332,-0.245287,-0.017752,-0.063630,0.029504,0.057136,...,-0.010247,-0.003016,0.003884,-0.176165,-0.169017,-0.185741,1.000000,-0.206622,-0.173273,-0.148881
dayofweek_4,0.125295,0.161771,0.187233,0.186716,0.163540,-0.262756,0.044577,-0.160692,0.044453,0.154072,...,0.000863,-0.000658,0.005729,-0.188711,-0.181055,-0.198969,-0.206622,1.000000,-0.185613,-0.159484
dayofweek_5,0.832906,0.095960,0.130209,0.133576,0.095222,0.706411,0.086875,-0.118097,0.046748,0.108937,...,-0.008602,-0.002406,0.000254,-0.158254,-0.151833,-0.166856,-0.173273,-0.185613,1.000000,-0.133743


In [12]:
df_corr.to_csv('corr.csv')

In [16]:
print(df_corr.Class.sort_values())

ADID_type_1   -0.523409
Platform_1    -0.254604
DSP_ID_3      -0.198255
Adunit_ID_1   -0.127484
Media_ID_1    -0.127484
                 ...   
Adunit_ID_2    0.234965
Platform_3     0.275789
DSP_ID_6       0.300603
ADID_type_5    0.569818
Class          1.000000
Name: Class, Length: 120, dtype: float64


In [17]:
np.abs(df_corr).

,Ex_Rate,P1,P2,P3,P4,weekend,Class,ADID_type_1,ADID_type_2,ADID_type_5,...,hour_21,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
Ex_Rate,1.000000,0.125311,0.158679,0.161411,0.125189,0.702662,0.111406,0.140940,0.050147,0.131556,...,0.009760,0.000003,0.004495,0.057682,0.323480,0.564041,0.153584,0.125295,0.832906,0.048624
P1,0.125311,1.000000,0.848253,0.856646,0.999616,0.003446,0.155970,0.527145,0.284579,0.459666,...,0.019302,0.022452,0.010754,0.088841,0.112484,0.057108,0.076163,0.161771,0.095960,0.112570
P2,0.158679,0.848253,1.000000,0.986888,0.851888,0.018168,0.148661,0.579750,0.344046,0.495672,...,0.020535,0.015989,0.005096,0.130321,0.133463,0.066688,0.091893,0.187233,0.130209,0.120761
P3,0.161411,0.856646,0.986888,1.000000,0.859313,0.021331,0.181787,0.618393,0.286140,0.554642,...,0.020707,0.015432,0.006840,0.131439,0.137037,0.065392,0.091819,0.186716,0.133576,0.120115
P4,0.125189,0.999616,0.851888,0.859313,1.000000,0.004175,0.153553,0.525342,0.284414,0.457849,...,0.019567,0.022805,0.010918,0.089320,0.112948,0.057422,0.076332,0.163540,0.095222,0.112761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dayofweek_2,0.564041,0.057108,0.066688,0.065392,0.057422,0.236202,0.074930,0.054116,0.018063,0.050425,...,0.011743,0.003865,0.002898,0.169640,0.162757,1.000000,0.185741,0.198969,0.166856,0.143366
dayofweek_3,0.153584,0.076163,0.091893,0.091819,0.076332,0.245287,0.017752,0.063630,0.029504,0.057136,...,0.010247,0.003016,0.003884,0.176165,0.169017,0.185741,1.000000,0.206622,0.173273,0.148881
dayofweek_4,0.125295,0.161771,0.187233,0.186716,0.163540,0.262756,0.044577,0.160692,0.044453,0.154072,...,0.000863,0.000658,0.005729,0.188711,0.181055,0.198969,0.206622,1.000000,0.185613,0.159484
dayofweek_5,0.832906,0.095960,0.130209,0.133576,0.095222,0.706411,0.086875,0.118097,0.046748,0.108937,...,0.008602,0.002406,0.000254,0.158254,0.151833,0.166856,0.173273,0.185613,1.000000,0.133743
